# 示例：RNN手写数字识别

### 1、数据导入

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(35)
mnist = input_data.read_data_sets('/home/chineseNER/jupyter/Tensorflow入门/dataset/MNIST_data', one_hot=True)

Extracting /home/chineseNER/jupyter/Tensorflow入门/dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/chineseNER/jupyter/Tensorflow入门/dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/chineseNER/jupyter/Tensorflow入门/dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/chineseNER/jupyter/Tensorflow入门/dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


### 2、数据观察

In [2]:
#mnist数据是55000张图片，每张图片是28*28 = 784像素
print ('输入数据的shape:')
print(mnist.train.images.shape) 
print('输入数据:') 
print (mnist.train.images)
print('单张图片输入数据:') 
print (mnist.train.images[0])

输入数据的shape:
(55000, 784)
输入数据:
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
单张图片输入数据:
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.  

### 3、设置模型参数

In [3]:
# hyperparameters设置模型参数，该参数是确定模型训练方式的
lr = 0.001
training_iters = 100000
batch_size = 128

n_inputs = 28   # MNIST data input (img shape: 28*28)28维
n_steps = 28    # time steps28行，每次读一行数据训练
n_hidden_units = 128   # neurons in hidden layer
n_classes = 10      # MNIST classes (0-9 digits)

### 4、定义模型变量和参数初始化

In [4]:
# tf Graph input，placehoder定义变量，hold住变量，在执行节点时传入数据给该变量
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) #none表示输入图片张数不确定
y = tf.placeholder(tf.float32, [None, n_classes])
# 初始化参数，该参数在模型训练过程中会不断更新，最后达到最优
weights = {
    # 生成维度是(28, 128)的标准正太分布，mean=0，stddev=1
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}
print(weights)
print(biases)

{'in': <tf.Variable 'Variable:0' shape=(28, 128) dtype=float32_ref>, 'out': <tf.Variable 'Variable_1:0' shape=(128, 10) dtype=float32_ref>}
{'in': <tf.Variable 'Variable_2:0' shape=(128,) dtype=float32_ref>, 'out': <tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>}


### 5、定义RNN神经单元

**补充知识：数据转换**

数据在进入RNN前会做一定的变换，这里先以下面例子说明数据变换过程。

In [5]:
#从下面几个小实验中可以看到接下来几个函数的效果
import numpy as np
a= np.asarray(range(20))
b = a.reshape(-1,2,2) #-1表示该维度数据没有被设置，函数会自行计算
c = b.reshape(-1,2)
print ('生成一列数据')
print(a) 
print('reshape(-1,2,2)函数的效果') 
print(b) 
print('b.shape:',b.shape)
print('reshape(-1,2)函数的效果') 
print(c) 
print('c.shape:',c.shape)

生成一列数据
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
reshape(-1,2,2)函数的效果
[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]

 [[12 13]
  [14 15]]

 [[16 17]
  [18 19]]]
b.shape: (5, 2, 2)
reshape(-1,2)函数的效果
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]]
c.shape: (10, 2)


In [6]:
e = np.transpose(b,[1,0,2]) #三个数字分别代表三个维度的index
print('transpose函数的效果') 
print(e)
print('e.shape:',e.shape)

transpose函数的效果
[[[ 0  1]
  [ 4  5]
  [ 8  9]
  [12 13]
  [16 17]]

 [[ 2  3]
  [ 6  7]
  [10 11]
  [14 15]
  [18 19]]]
e.shape: (2, 5, 2)


In [7]:
def RNN(X, weights, biases):
    # 输入层
    ########################################

    # transpose the inputs shape from
    # (128 batch ,28 steps, 28 inputs) ==> (128 batch * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])

    # into hidden
    # X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

    # 隐藏层
    ##########################################

    # BasicLSTMCell定义单个cell
    cell = tf.contrib.rnn.BasicRNNCell(n_hidden
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    #dynamic_rnn函数将cell连成RNN 网络
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state)

    # 输出层：输出logits
    #############################################
    #results = tf.matmul(final_state[1], weights['out']) + biases['out']
    
    # unpack to list [(batch, outputs)..] * steps
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul([-1], weights['out']) + biases['out']    # shape = (128, 10),每张图片10个logit值

    return results

### 6、定义损失函数、优化方法

**补充知识：tf.argmax函数**

tf.argmax是tensorflow使用numpy api实现的np.argmax，原理与np.argmax一致。函数使用方式有tf.argmax(array, 1)和tf.argmax(array, 0)，其中0/1是axis，axis是为了方便不同维度数据的计算，以下用例子说明。

In [8]:
import numpy as np
test = np.array([[1, 2, 3], [2, 3, 4], [5, 4, 3], [8, 7, 2]])

* 当axis=0时，函数取各维度的最大值的位置

In [9]:
np.argmax(test,0)

array([3, 3, 1])

* 当axis=1时，函数取各样本的最大值的位置

In [10]:
np.argmax(test,1)

array([2, 2, 0, 0])

In [11]:
#损失函数是整个minibatch的目标类别和预测类别之间的交叉熵,
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### 7、输入数据，运行模型

In [12]:
with tf.Session() as sess:
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print('step:',step,'accuracy:',sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
            }))
        step += 1

step: 0 accuracy: 0.210938
step: 20 accuracy: 0.5
step: 40 accuracy: 0.460938
step: 60 accuracy: 0.53125
step: 80 accuracy: 0.546875
step: 100 accuracy: 0.546875
step: 120 accuracy: 0.5625
step: 140 accuracy: 0.546875
step: 160 accuracy: 0.484375
step: 180 accuracy: 0.53125
step: 200 accuracy: 0.515625
step: 220 accuracy: 0.664062
step: 240 accuracy: 0.609375
step: 260 accuracy: 0.585938
step: 280 accuracy: 0.65625
step: 300 accuracy: 0.5625
step: 320 accuracy: 0.726562
step: 340 accuracy: 0.585938
step: 360 accuracy: 0.578125
step: 380 accuracy: 0.65625
step: 400 accuracy: 0.695312
step: 420 accuracy: 0.648438
step: 440 accuracy: 0.617188
step: 460 accuracy: 0.53125
step: 480 accuracy: 0.679688
step: 500 accuracy: 0.648438
step: 520 accuracy: 0.6875
step: 540 accuracy: 0.640625
step: 560 accuracy: 0.59375
step: 580 accuracy: 0.734375
step: 600 accuracy: 0.671875
step: 620 accuracy: 0.671875
step: 640 accuracy: 0.703125
step: 660 accuracy: 0.578125
step: 680 accuracy: 0.6875
step: 700 

**参考文献**    
例子：https://github.com/MorvanZhou/tutorials/tree/master/tensorflowTUT/tf20_RNN2   
argmax：http://blog.csdn.net/qq575379110/article/details/70538051   
数据转换：https://github.com/FanGhost/RNNStudy/blob/master/simpleRNN.ipynb    